In [2]:
# Import libraries
import pandas as pd
import numpy as np

In [3]:
# Configuration: File path and exchange rate
EXCEL_FILE = "jp.xlsx"
JPY_TO_USD = 1/145.0

In [4]:
# Add Category_Type and Country columns
def add_metadata(df):
    def assign_category_type(category):
        if category == "All Employees":
            return "Total"
        elif category in ["Male", "Female"]:
            return "Gender"
        else:
            return "Industry"
    
    df['Category_Type'] = df['Category'].apply(assign_category_type)
    df["Country"] = "Japan"
    
    return df[["Country", "Category", "Category_Type", "Education", "Salary", "Population"]].reset_index(drop=True)

In [5]:
# Filter and process categories
def process_categories(df, education_label):
    # Categories to keep
    keep_categories = [
        "All Employees", "Male", "Female",
        "Agriculture, Forestry, Fishing & Animal Husbandry",
        "Mining & Quarrying", "Manufacturing",
        "Electricity & Gas Supply", "Water Supply & Waste Management",
        "Construction", "Wholesale & Retail Trade",
        "Transportation & Storage", "Accommodation & Food Service",
        "Publishing, Audio-Visual & Information Communication",
        "Finance & Insurance", "Real Estate",
        "Professional, Scientific & Technical Services",
        "Education", "Health Care & Social Work",
        "Arts, Entertainment & Recreation",
        "Other Services", "Combined Services"
    ]
    
    # Filter and deduplicate
    df = df[df['Category'].isin(keep_categories)].copy()
    df = df.drop_duplicates(subset=['Category'], keep='first').copy()
    
    # Add education and salary
    df["Education"] = education_label
    df["Salary"] = df["SalaryJPYk"] * 1000 * JPY_TO_USD
    
    return df


In [6]:
# Utility: Auto-detect skiprows in Excel file
def detect_skiprows(df, probe_max=20):
    for i in range(min(probe_max, len(df))):
        temp_cols = df.iloc[i].astype(str).values
        if any("千円" in str(c) for c in temp_cols):
            return i
    return 10

In [7]:
# Utility: Select industry, salary, and population columns
def pick_cols(df):
    # Industry column
    candidates_ind = [c for c in df.columns if ("Unnamed: 1" in str(c)) or ("区 分" in str(c)) or ("産業" in str(c))]
    if not candidates_ind:
        candidates_ind = [df.columns[0]]
    industry_col = candidates_ind[0]
    
    # Salary column
    candidates_sal = [c for c in df.columns if "千円" in str(c)]
    if not candidates_sal:
        raise ValueError("Cannot find salary column")
    salary_col = candidates_sal[0]
    
    # Population column
    candidates_pop = [c for c in df.columns if "十人" in str(c)]
    population_col = candidates_pop[0] if candidates_pop else None
    
    return industry_col, salary_col, population_col


In [8]:
# Configuration: Japanese to English category mapping
INDUSTRY_MAP = {
    "男女計\n産業計": "All Employees", "男女計\n 産業計": "All Employees", "男女計 産業計": "All Employees", "産業計": "All Employees",
    "男\n産業計": "Male", "男\n 産業計": "Male",
    "女\n産業計": "Female", "女\n 産業計": "Female",
    "Ｃ 鉱業，採石業，砂利採取業": "Mining & Quarrying",
    "Ｄ 建設業": "Construction",
    "Ｅ 製造業": "Manufacturing",
    "Ｇ 情報通信業": "Publishing, Audio-Visual & Information Communication",
    "Ｈ 運輸業，郵便業": "Transportation & Storage",
    "Ｉ 卸売業，小売業": "Wholesale & Retail Trade",
    "Ｊ 金融業，保険業": "Finance & Insurance",
    "Ｋ 不動産業，物品賃貸業": "Real Estate",
    "Ｌ 学術研究，専門・技術ｻｰﾋﾞｽ業": "Professional, Scientific & Technical Services",
    "Ｍ 宿泊業，飲食ｻｰﾋﾞｽ業": "Accommodation & Food Service",
    "Ｏ 教育，学習支援業": "Education",
    "Ｐ 医療，福祉": "Health Care & Social Work",
    "Ｒ ｻｰﾋﾞｽ業(他に分類されないもの)": "Other Services",
    "Ｆ 電気・ガス・熱供給・水道業": "Electricity & Gas Supply",
    "Ｎ 生活関連ｻｰﾋﾞｽ業，娯楽業": "Arts, Entertainment & Recreation",
    "Ｑ 複合ｻｰﾋﾞｽ事業": "Combined Services",
}

In [9]:
# Main: Read and clean one Excel sheet
def clean_one_sheet(sheet_name, education_label):
    # Read Excel file
    raw = pd.read_excel(EXCEL_FILE, sheet_name=sheet_name, header=None)
    skip = detect_skiprows(raw)
    df_raw = pd.read_excel(EXCEL_FILE, sheet_name=sheet_name, skiprows=skip)
    
    # Get columns
    ind_col, sal_col, pop_col = pick_cols(df_raw)
    cols_to_select = [ind_col, sal_col]
    if pop_col:
        cols_to_select.append(pop_col)
    
    df = df_raw[cols_to_select].copy()
    
    # Rename columns
    new_col_names = ["Category", "SalaryJPYk"]
    if pop_col:
        new_col_names.append("Population10s")
    df.columns = new_col_names
    
    # Clean salary - fixed to avoid FutureWarning
    df["SalaryJPYk"] = pd.to_numeric(df["SalaryJPYk"].replace(["-", "－", "—"], np.nan), errors="coerce")
    
    # Clean population - fixed to avoid FutureWarning
    if pop_col:
        df["Population10s"] = pd.to_numeric(df["Population10s"].replace(["-", "－", "—"], np.nan), errors="coerce")
        df["Population"] = df["Population10s"] * 10
    else:
        df["Population"] = np.nan
    
    # Remove empty categories
    df = df[df["Category"].notna()].copy()
    
    # Translate categories
    df["Category"] = df["Category"].replace(INDUSTRY_MAP)
    
    return df


In [10]:
# Configuration: Sheet names for each education level
SHEETS = [
    {"education": "Junior High",     "sheet_names": ["中学"]},
    {"education": "Senior High",     "sheet_names": ["高校"]},
    {"education": "Junior College",  "sheet_names": ["高専・短大", "専門学校"]},
    {"education": "University",      "sheet_names": ["大学"]},
    {"education": "Graduate School", "sheet_names": ["大学院"]},
]


In [11]:
# Process all sheets and combine into long format
long_list = []
for s in SHEETS:
    education_label = s["education"]
    sheet_names = s["sheet_names"]
    education_data_list = []
    
    for sheet_name in sheet_names:
        try:
            df = clean_one_sheet(sheet_name, education_label)
            df = process_categories(df, education_label)
            df = add_metadata(df)
            education_data_list.append(df)
        except Exception as e:
            print(f"[WARN] Failed: {education_label} - {sheet_name}: {e}")
    
    if education_data_list:
        df_education = pd.concat(education_data_list, ignore_index=True)
        long_list.append(df_education)

df_jp_long = pd.concat(long_list, ignore_index=True) if long_list else pd.DataFrame()

C:\Users\chen.yi.chieh\AppData\Local\Temp\ipykernel_31192\1398371368.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["SalaryJPYk"] = pd.to_numeric(df["SalaryJPYk"].replace(["-", "－", "—"], np.nan), errors="coerce")
C:\Users\chen.yi.chieh\AppData\Local\Temp\ipykernel_31192\1398371368.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Population10s"] = pd.to_numeric(df["Population10s"].replace(["-", "－", "—"], np.nan), errors="coerce")
C:\Users\chen.yi.chieh\AppData\Local\Temp\ipykernel_31192\1398371368.py:23: FutureWa

In [12]:
# Configuration: Category and education ordering
education_order = ["Junior High", "Senior High", "Junior College", "University", "Graduate School"]
desired_category_order = [
    "All Employees",
    "Male",
    "Female",
    "Small & Medium Enterprises",
    "Large Enterprises & Others",
    "Agriculture, Forestry, Fishing & Animal Husbandry",
    "Mining & Quarrying",
    "Manufacturing",
    "Electricity & Gas Supply",
    "Water Supply & Waste Management",
    "Construction",
    "Wholesale & Retail Trade",
    "Transportation & Storage",
    "Accommodation & Food Service",
    "Publishing, Audio-Visual & Information Communication",
    "Finance & Insurance",
    "Real Estate",
    "Professional, Scientific & Technical Services",
    "Administrative & Support Services",
    "Education",
    "Health Care & Social Work",
    "Arts, Entertainment & Recreation",
    "Other Services",
]

# Sort df_jp_long by education and category order
education_order_map = {edu: i for i, edu in enumerate(education_order)}
category_order_map = {cat: i for i, cat in enumerate(desired_category_order)}

df_jp_long['edu_order'] = df_jp_long['Education'].map(education_order_map)
df_jp_long['cat_order'] = df_jp_long['Category'].map(category_order_map)

df_jp_long = df_jp_long.sort_values(['edu_order', 'cat_order']).drop(['edu_order', 'cat_order'], axis=1).reset_index(drop=True)

In [13]:
# Add 'Real_Salary' column for Japan using PPP factor 93.21
df_jp_long['Real Salary'] = df_jp_long['Salary']* 145 / 93.21
df_jp_long

,Country,Category,Category_Type,Education,Salary,Population,Real Salary
0,Japan,All Employees,Total,Junior High,1522.068966,400.0,2367.771698
1,Japan,Male,Gender,Junior High,1586.896552,200.0,2468.619247
2,Japan,Female,Gender,Junior High,1455.172414,190.0,2263.705611
3,Japan,Mining & Quarrying,Industry,Junior High,NaN,NaN,NaN
4,Japan,Manufacturing,Industry,Junior High,1452.413793,160.0,2259.414226
...,...,...,...,...,...,...,...
109,Japan,Education,Industry,Graduate School,2184.137931,1040.0,3397.704109
110,Japan,Health Care & Social Work,Industry,Graduate School,1644.137931,500.0,2557.665487
111,Japan,"Arts, Entertainment & Recreation",Industry,Graduate School,1700.689655,220.0,2645.638880
112,Japan,Other Services,Industry,Graduate School,1664.827586,1030.0,2589.850874


In [ ]:
# Export long format to CSV
df_jp_long.to_csv('data/df_jp_long.csv', index=False)
print("df_jp_long.csv saved")


df_jp_long.csv saved


In [15]:
# Create wide format with weighted averages
# Calculate weighted average for each Category-Education combination
agg_data = df_jp_long.groupby(['Category', 'Education']).agg({
    'Salary': lambda x: (x * df_jp_long.loc[x.index, 'Population']).sum() / df_jp_long.loc[x.index, 'Population'].sum(),
    'Population': 'sum',
    'Category_Type': 'first'
}).reset_index()

# Pivot to wide format
wide = agg_data.pivot_table(index='Category', columns='Education', values='Salary').reset_index()
wide.columns.name = None

# Ensure all education columns exist
need_edu_cols = education_order
for edu_col in need_edu_cols:
    if edu_col not in wide.columns:
        wide[edu_col] = np.nan

ordered_cols = ['Category'] + need_edu_cols
wide = wide[ordered_cols].copy()

# Reorder rows by desired category order
wide_reordered = wide.set_index('Category').reindex(desired_category_order).reset_index()

# Add Category_Type
category_type_map = agg_data.set_index('Category')['Category_Type'].to_dict()
wide_reordered['Category_Type'] = wide_reordered['Category'].map(category_type_map)

# Reorder columns to put Category_Type after Category
cols = wide_reordered.columns.tolist()
if 'Category_Type' in cols:
    cols.insert(cols.index('Category') + 1, cols.pop(cols.index('Category_Type')))
    wide_reordered = wide_reordered[cols]

C:\Users\chen.yi.chieh\AppData\Local\Temp\ipykernel_31192\1010094785.py:4: RuntimeWarning: invalid value encountered in scalar divide
  'Salary': lambda x: (x * df_jp_long.loc[x.index, 'Population']).sum() / df_jp_long.loc[x.index, 'Population'].sum(),


In [16]:
wide_reordered

,Category,Category_Type,Junior High,Senior High,Junior College,University,Graduate School
0,All Employees,Total,1522.068966,1362.068966,1538.810605,1712.413793,1982.068966
1,Male,Gender,1586.896552,1382.758621,1532.229083,1733.103448,2001.379310
2,Female,Gender,1455.172414,1322.068966,1542.051387,1688.965517,1917.931034
3,Small & Medium Enterprises,NaN,NaN,NaN,NaN,NaN,NaN
4,Large Enterprises & Others,NaN,NaN,NaN,NaN,NaN,NaN
5,"Agriculture, Forestry, Fishing & Animal Husbandry",NaN,NaN,NaN,NaN,NaN,NaN
6,Mining & Quarrying,Industry,NaN,1402.068966,1831.724138,2226.896552,2297.241379
7,Manufacturing,Industry,1452.413793,1354.482759,1444.830902,1694.482759,1967.586207
8,Electricity & Gas Supply,Industry,NaN,1306.206897,1484.545455,1660.000000,1833.103448
9,Water Supply & Waste Management,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Export wide format to CSV
wide_reordered.to_csv('data/df_jp_wide.csv', index=False)
print("df_jp_wide.csv saved")


df_jp_wide.csv saved
